In [1]:
import os
import pandas as pd
import re
import json
from nltk.corpus import stopwords
import nltk
import spacy


nltk.download('punkt')
nltk.download('stopwords')


class TextPreprocessor:
    def __init__(self):
        self.stop_phrases = [
            'Ещё больше новостей — в телеграм-канале Москва 24 Подписывайтесь!', 
            'Подробнее – в эфире телеканала Москва 24.', 
            'РИА «Сахалин-Курилы»'
        ]

        self.tags_keyword = '\nTags: '

        self.word_with_capital_letter_pattern = re.compile('^[А-ЯA-Z]')
        self.sentence_ending_char_on_new_line_pattern = re.compile('(?<=[а-я])\n(?=\\.)')
        self.footnote_pattern = re.compile('^\\*+')
        self.citation_beginning_pattern = re.compile('^("|“|«)')
        self.two_spaces_pattern = re.compile('\\s{2,}')
        self.two_quotes_pattern = re.compile('"{2,}')
        self.ria_news_reference_pattern = re.compile('\\b[А-Я]+, \\d{1,2} [а-я]+ — РИА Новости\\b')
        self.website_pattern = re.compile('\\b(https?://)?[a-zA-Z\\d][\\w\\-]*(\\.[a-zA-Z\\d][\\w\\-]*[a-zA-Z\\d])*\\.[\\w\\-]*[a-zA-Z\\d]\\b')
        self.visual_credit_pattern = re.compile('\\b(Видео|Фото): .+$', re.IGNORECASE)

        self.stop_words = stopwords.words('russian')
        self.stop_words.extend(['это', 'эта', 'этот', 'ещё', 'которые', 'который', 
                                'которым', 'также', 'её', 'мочь', 'год', 'февраль'])

        # We apply the SpaCy NLP pipeline trained on russian language for lemmatization
        self.russian_nlp = spacy.load('ru_core_news_lg')


    def _preprocess_tags(self, tags: list[str]):
        _tags = [t.lower().replace('#', '') for t in tags]

        _tags = [t for t in _tags if len(t.split(' ')) == 1]

        return _tags
    

    def _tokenize_text(self, text: str):
        document = self.russian_nlp(text)

        tokens = []

        for token in document:
            t = token.lemma_.lower()

            if (t in self.stop_words) or not t.isalpha():
                continue

            tokens.append(t)

        return tokens


    def preprocess_text(self, text: str):
        # There is a key word "Tags:" in some news items. This word marks 
        # the beginning of a sequence of tags that were assigned to a corresponding 
        # news message. These tags are irrelevant for the topic modeling of the corpus, so we delete them
        text = text.strip('"')
        text = re.sub(self.two_quotes_pattern, '"', text)

        if self.tags_keyword in text:
            _text, tags_sequence = text.split(self.tags_keyword)

            tags = [t.strip(' ').replace('#', '') for t in tags_sequence.split(',')]

            # We preprocess tags to use them later as 
            # ground truth in the topic modeling quality estimation
            tags = self._preprocess_tags(tags)
        else:
            _text = text + ''

            tags = []

        lines = _text.split('\n')

        relevant_lines = []
        footnotes = []

        for l in lines:
            l = l.strip(' \t')

            l = re.sub(self.visual_credit_pattern, '', l)

            if len(l) == 0:
                continue
            
            # We identify lines of text that are footnotes and set them aside of a news message text
            if bool(self.footnote_pattern.search(l)):
                footnotes.append(l)
            else:
                relevant_lines.append(l)

        _text = ' '.join(relevant_lines)
        _text = re.sub(self.two_spaces_pattern, ' ', _text)

        # We delete stop phrases from the text
        for phi in self.stop_phrases:
            _text = _text.replace(phi, '')

        _text = re.sub(self.ria_news_reference_pattern, '', _text)
        _text = re.sub(self.website_pattern, '', _text)

        _text = _text.strip()

        tokens = self._tokenize_text(_text)

        return _text, tokens, tags, footnotes
    

tatar_specific_characters = set('ӘәҖҗӨөҢңҺһ')


def tatar_characters_present(text: str):
    text_char_set = set(text)

    return len(tatar_specific_characters & text_char_set) > 0

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mdmytriiev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mdmytriiev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
root_path = 'datasets'

filename = 'test_assignment_data.csv'

raw_data = pd.read_csv(os.path.join(root_path, filename))

preprocessor = TextPreprocessor()

X = []
processed_texts = []
processed_dataset = []

for i, row in raw_data.iterrows():
    if tatar_characters_present(row['fullText']):
        continue
    
    text, tokens, tags, footnotes = preprocessor.preprocess_text(row['fullText'])

    processed_dataset.append({
        'text': text, 
        'tokens': tokens, 
        'tags': tags, 
        'footnotes': footnotes
    })

    processed_texts.append(text)
    X.append(tokens)

with open(os.path.join(root_path, 'processed_dataset.json'), 'w') as f:
    json.dump(processed_dataset, f)

In [3]:
# It could be seen that all news in the corpus were published in February, so name of the month is irrelevant 
years = set()
months = set()
days = set()

k = 0

for _, row in raw_data.iterrows():
    if str(row['pubTime']) == 'nan':
        k += 1

        continue

    y, m, d = row['pubTime'].split('-')

    years.add(y)
    months.add(m)
    days.add(d)

print('Years:', years)
print('Months:', months)
print('Days:', days)
print('Fraction of entries with unknown publication date:', k / len(raw_data))

Years: {'2024'}
Months: {'02'}
Days: {'21', '20', '19'}
Fraction of entries with unknown publication date: 0.0


In [4]:
processed_texts[:10]

['Бабушкинский суд столицы приговорил трех активистов движения "СтопХам" к шести годам тюремного заключения каждого за драку с сотрудниками спецподразделения "Гром" в Москве. Об этом сообщается в телеграм-канале судов общей юрисдикции столицы.Там уточнили, что суд признал виновными Кирилла Бунина, Кирилла Котова и Алексея Горбачевского. Они обвиняются в применении насилия в отношении представителя власти (статья 318 УК РФ) и в совершенном группой лиц хулиганстве (часть 2 статьи 213 УК РФ). Отбывать наказание мужчины будут в колонии общего режима, добавили там. Конфликт между активистами и правоохранителями произошел на Ярославском шоссе 23 июня 2022 года из-за неправильно припаркованной машины сотрудников спецотряда. Отмечалось, что обвиняемые ругались и били по капоту машины, после чего началась драка.Спустя два дня Бабушкинский суд Москвы арестовал нападавших до 23 августа 2022 года. В июле 2023-го в столице начался процесс над активистами "СтопХама".',
 'Бывшего совладельца сетей "К

In [5]:
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvisualize

In [6]:
# We create a dictionary of the corpus
dictionary = corpora.Dictionary(X)
dictionary.filter_extremes(no_below=5, no_above=0.9)

# We transform our corpus to the bag-of-words
corpus = [dictionary.doc2bow(tokens_list) for tokens_list in X]

num_topics = 50

# train LDA model
lda_model = LdaModel(corpus=corpus, 
                     id2word=dictionary, 
                     random_state=4583, 
                     chunksize=200, 
                     num_topics=num_topics, 
                     passes=200, 
                     iterations=400)

# Print LDA topics as sequences of ten words that enter each topic with the highest probability
for topic in lda_model.print_topics(num_topics=num_topics, num_words=10):
    print(topic)

lda_model.save('topic_model/lda_model')

(0, '0.041*"производство" + 0.034*"конкурс" + 0.032*"предприятие" + 0.031*"культура" + 0.027*"герой" + 0.025*"фестиваль" + 0.024*"тема" + 0.021*"история" + 0.018*"направление" + 0.017*"продукция"')
(1, '0.136*"украина" + 0.081*"сша" + 0.039*"заявить" + 0.036*"американский" + 0.029*"киев" + 0.028*"страна" + 0.027*"ранее" + 0.023*"нато" + 0.021*"президент" + 0.019*"помощь"')
(2, '0.097*"губернатор" + 0.040*"лицензия" + 0.038*"участок" + 0.035*"реализация" + 0.034*"фонд" + 0.034*"округ" + 0.031*"доллар" + 0.029*"актив" + 0.027*"деньга" + 0.026*"финансирование"')
(3, '0.019*"право" + 0.017*"мера" + 0.016*"закон" + 0.015*"гражданин" + 0.014*"нарушение" + 0.013*"направить" + 0.012*"документ" + 0.012*"средство" + 0.012*"необходимый" + 0.011*"условие"')
(4, '0.073*"язык" + 0.072*"военнослужащий" + 0.053*"начальник" + 0.053*"руководство" + 0.046*"британский" + 0.043*"исследование" + 0.038*"подразделение" + 0.038*"игорь" + 0.035*"китайский" + 0.025*"актриса"')
(5, '0.062*"операция" + 0.059*"спец

In [21]:
with open('datasets/processed_dataset.json', 'r') as f:
    processed_dataset = json.load(f)

X = [item['tokens'] for item in processed_dataset]

In [8]:
coherence_model = CoherenceModel(model=lda_model, texts=X, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(coherence_score)

0.4502994839800708


In [9]:
russian_news_topics_visualization = gensimvisualize.prepare(lda_model, corpus, dictionary, mds='mmds')
pyLDAvis.display(russian_news_topics_visualization)